# Análise das variáveis/indicadores | Regressão Linear

- Apenas variáveis de sono & todas as formas de bullying

In [1]:
import pandas as pd
import seaborn as sns
import plotly.express as px

In [2]:
# carrega dataset pré-processado
df_train = pd.read_csv('../../../datasets/new-bullying_notamedia_train.csv', sep=',')
df_test = pd.read_csv('../../../datasets/new-bullying_notamedia_test.csv', sep=',')

In [3]:
# mostre todas as variáveis
df_train.columns

Index(['Unnamed: 0', 'Escola', 'Curso', 'Cor ', 'Genero', 'Ano', 'Idade',
       'Demora dormir', 'Sonhos ruins', 'Qualidade do sono',
       'Relação com Amigos', 'Relação com Professores', 'Auto-Imag', 'Fisico',
       'Social', 'Verbal', 'Material', 'Virtual', 'Mora com pais',
       'Escolari-mãe', 'Escolari-pai', 'Nota-exatas', 'Nota-humanas',
       'Nota-biológicas', 'Tipo escola', 'Bullying Total', 'Sofreu Bullying?',
       'Cor Agrupada', 'Média-notas', 'Média-notas-cat', 'SOFREU-BULLYING'],
      dtype='object')

In [4]:
variaveis_numericas = ['Fisico', 'Social', 'Verbal', 'Material', 'Virtual']
variaveis_categoricas = ['Demora dormir', 'Sonhos ruins', 'Qualidade do sono']
variavel_target = 'Média-notas'

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
    ('robust_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('one-hot-encoding', OneHotEncoder(handle_unknown='ignore'))
])


# (name, transformer, columns)
preprocessed_pipeline = ColumnTransformer([
    ('numerical', num_pipeline, variaveis_numericas),
    ('categorical', cat_pipeline, variaveis_categoricas)
])

In [6]:
# treina o modelo de pré-processamento e
# preprocessa o treinamento
X_train = preprocessed_pipeline.fit_transform(df_train)
X_train

array([[-0.55842386, -0.73434002, -0.90016377, ...,  1.        ,
         0.        ,  0.        ],
       [-0.55842386,  1.05647254,  1.12844738, ...,  0.        ,
         0.        ,  0.        ],
       [-0.55842386, -0.73434002,  0.1141418 , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.55842386, -0.73434002, -0.90016377, ...,  0.        ,
         0.        ,  0.        ],
       [-0.55842386, -0.73434002, -0.90016377, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.04704473, -0.73434002, -0.90016377, ...,  0.        ,
         1.        ,  0.        ]])

In [7]:
X_train.min(axis=0), X_train.max(axis=0)

(array([-0.55842386, -0.73434002, -0.90016377, -0.63208905, -0.48104998,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ]),
 array([3.45524761, 2.84728511, 2.14275296, 3.46426708, 4.30091292,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        ]))

In [8]:
# recupera os nomes das features após aplicar o one-hot encoding
cat_feature_names = preprocessed_pipeline.named_transformers_['categorical'].named_steps['one-hot-encoding'].get_feature_names()
cat_feature_names

c:\users\bianca\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['x0_1-2 vezes por semana', 'x0_2-3 vezes por semana',
       'x0_< 1 vez por semana', 'x0_> 3 vezes por semana', 'x1_0', 'x1_1',
       'x1_2', 'x1_3', 'x2_Boa', 'x2_Muito boa', 'x2_Muito ruim',
       'x2_Regular', 'x2_Ruim'], dtype=object)

In [9]:
feature_names = variaveis_numericas + list(cat_feature_names)
feature_names

['Fisico',
 'Social',
 'Verbal',
 'Material',
 'Virtual',
 'x0_1-2 vezes por semana',
 'x0_2-3 vezes por semana',
 'x0_< 1 vez por semana',
 'x0_> 3 vezes por semana',
 'x1_0',
 'x1_1',
 'x1_2',
 'x1_3',
 'x2_Boa',
 'x2_Muito boa',
 'x2_Muito ruim',
 'x2_Regular',
 'x2_Ruim']

In [10]:
len(feature_names)

18

In [11]:
# notas médias do treinamento
y_train = df_train['Média-notas']
y_train

0      3.333333
1      4.000000
2      3.333333
3      4.000000
4      4.000000
         ...   
386    4.000000
387    3.333333
388    4.000000
389    3.333333
390    3.666667
Name: Média-notas, Length: 391, dtype: float64

In [12]:
# preprocessa o teste
X_test = preprocessed_pipeline.transform(df_test)
y_test = df_test['Média-notas']

In [13]:
# treinando um regressor linear
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(X_train, y_train)

LinearRegression()

In [14]:
# predição do conjunto de treinamento
y_train_pred = regressor.predict(X_train)

In [15]:
# coeficiente de determinação
# quanto mais próximo de 1, significa que as variáveis de estudo explicam melhor a variável de saída
from sklearn.metrics import r2_score

r2_train = r2_score(y_train, y_train_pred)
print(f'Coeficiente de Determinação para o Treinamento: {r2_train}')

Coeficiente de Determinação para o Treinamento: 0.07773973247276167


In [16]:
# medindo o erro de predição para o treinamento (que são amostras já vistas durante o treinamento)
from sklearn.metrics import mean_absolute_error

mae_train = mean_absolute_error(y_train, y_train_pred)
print(f'Mean Absolute Error para o Treinamento: {mae_train}')

Mean Absolute Error para o Treinamento: 0.4846894763566541


In [17]:
# predição do conjunto de treinamento
y_test_pred = regressor.predict(X_test)

In [18]:
# medindo o erro de predição para o teste
from sklearn.metrics import mean_absolute_error

mae_test = mean_absolute_error(y_test, y_test_pred)
print(f'Mean Absolute Error para o Teste: {mae_test}')

Mean Absolute Error para o Teste: 0.4890695519389379


In [19]:
from sklearn.metrics import mean_squared_error

rmse = mean_squared_error(y_test, y_test_pred, squared=False)
print(f'RMSE para o Teste: {rmse}')

RMSE para o Teste: 0.6164619140430705


In [20]:
# visualizando quais são as variáveis mais importantes/relevantes para a predição
import eli5

In [21]:
eli5.show_weights(regressor, feature_names=feature_names, top=100)

Weight?,Feature
+3.712,<BIAS>
+0.099,x1_3
+0.097,x2_Regular
+0.093,x2_Ruim
+0.072,Material
+0.065,x0_1-2 vezes por semana
+0.038,x1_2
+0.036,Virtual
+0.019,x1_0
+0.008,x0_< 1 vez por semana


In [22]:
# quanto maior o valor, mais importante a feature é para o modelo

In [23]:
# features categóricas ==> para descobrir o que é x0, x1, etc
for i, feat_name in enumerate(variaveis_categoricas):
    print(f'x{i} ==> {feat_name}')

x0 ==> Demora dormir
x1 ==> Sonhos ruins
x2 ==> Qualidade do sono
